In [42]:
import pygrib
import numpy as np
import pickle
import os
from collections import OrderedDict

In [71]:
grbs = pygrib.open('A24560-201312.pgb.f00')
uin = grbs.select(shortName='u', typeOfLevel='isobaricInhPa', level = lambda l: l >= 100 and l <= 1000)
vin = grbs.select(shortName='v', typeOfLevel='isobaricInhPa', level = lambda l: l >= 100 and l <= 1000)
grbs.close()

Time = []
for i in range(len(uin)):
    u_wind_i = uin[i]
    v_wind_i = vin[i]

    lat_grid = u_wind_i.data(lat1 = 20, lat2 = 55, lon1 = 230, lon2 = 300)[1].flatten()
    lon_grid = u_wind_i.data(lat1 = 20, lat2 = 55, lon1 = 230, lon2 = 300)[2].flatten()
    lon_lat_grid = np.dstack((lon_grid, lat_grid))[0]
    
    pickle.dump(lon_lat_grid, open('LonLat_Grid.p','wb'), protocol=2)
    
    date_time = str(u_wind_i.year) + '_' + str(u_wind_i.month).zfill(2) + '_' + str(u_wind_i.day).zfill(2) + '_' + str(u_wind_i.hour).zfill(2) + '00'
    if date_time not in Time:
        if i != 0:
            pickle.dump(wind_data, open(date_time + '.p','wb'), protocol=2)
        wind_data = OrderedDict()
        Time.append(date_time)

        wind_data[u_wind_i.level] = np.dstack((u_wind_i.data(lat1 = 20, lat2 = 55, lon1 = 230, lon2 = 300)[0].flatten(),
                                                 v_wind_i.data(lat1 = 20, lat2 = 55, lon1 = 230, lon2 = 300)[0].flatten()))[0]
        
    else:
        wind_data[u_wind_i.level] = np.dstack((u_wind_i.data(lat1 = 20, lat2 = 55, lon1 = 230, lon2 = 300)[0].flatten(),
                                                 v_wind_i.data(lat1 = 20, lat2 = 55, lon1 = 230, lon2 = 300)[0].flatten()))[0]
        
    pickle.dump(wind_data, open(date_time + '.p','wb'), protocol=2) # m/s